In [5]:
import torch
import random
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

from syft.workers.websocket_client import WebsocketClientWorker

In [6]:

x = 25

def encrypt(x, Q = 1234567891011, shares = 3):
    share_a = random.randint(-Q,Q) # 21423462242
    share_b = random.randint(-Q,Q) # 32526344214
    share_c = (x - share_a - share_b) % Q # 2446313542
    return (share_a, share_b,  share_c)

agg1 = 21423462242 + 32526344214 + 2446313542 = 25
agg2 = 21456124565 + 21435465475 + 9876567876 = 25

(21423462242 + 32526344214 + 2446313542) + (21456124565 + 21435465475 + 9876567876)
= (21423462242+ 21456124565) + (32526344214 + 21435465475) + (2446313542 + 9876567876)

encrypt(x)
# 1. number of partitions 
# 2. share_a -> node 1, share_b -> node 2 , share_c -> node 3

SyntaxError: invalid syntax (<ipython-input-6-cd0527a220e1>, line 13)

## helpers

In [7]:
def send_data_to_worker(data, features, target, compute_nodes):
    remote_dataset = []
    X = torch.tensor(data[features].values, requires_grad=False).float()
    y = torch.from_numpy(data[target].values).float()

    train = TensorDataset(X, y)
    train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

    for batch_idx, (data,target) in enumerate(train_loader):
        data = data.send(compute_nodes)
        target = target.send(compute_nodes)
        remote_dataset.append((data, target))
    return remote_dataset

def update(data, target, model, optimizer):
    model.send(data.location)
    optimizer.zero_grad()
    pred = model(data)
    loss = nn.BCELoss()
    output = loss(pred.squeeze(), target)
    output.backward()
    optimizer.step()
    return model

def train(epoch, compute_nodes, remote_dataset, models, mpc_1, mpc_2):
    for data_index in range(len(remote_dataset[0])-1):
        # update remote models
        for remote_index in range(len(compute_nodes)):
            # remote_index = 0, alice
            # remote_index = 1, bob
            # alice -> data -> 400 <- data_index
            
            # models = [alice_model, bob_model]
            data, target = remote_dataset[remote_index][data_index]
            models[remote_index] = update(data, target, models[remote_index], optimizers[remote_index])
            # models = [trained_alice_model_on_alice_data, bob_model]
            
        # encrypted aggregation
        # global_params
        new_params = list()
        
        for param_i in range(len(params[0])):
            spdz_params = list()
            for remote_index in range(len(compute_nodes)):
                # remote_index = 0, alice
                # remote_index = 1, bob
                
                # trained_alice_model_on_alice_data -> paras -> (mpc_1, mpc_2)
                # trained_bob_model_on_bob_data -> paras -> (mpc_1, mpc_2)
                spdz_params.append(params[remote_index][param_i].fix_precision().share(mpc_1, mpc_2, crypto_provider=james).get())
            
            # spdz_params -> ((alice_mpc_1, alice_mpc_2), (bob_mpc_1, bob_mpc_2))
            # ((alice_mpc_1, alice_mpc_2) + (bob_mpc_1, bob_mpc_2))/2
            # (alice_mpc_1+bob_mpc_1)/2 (alice_mpc_2+bob_mpc_2)/2
            new_param = (spdz_params[0] + spdz_params[1]).get().float_precision()/2
            new_params.append(new_param)
        
        
        # assign global params to alice and bob
        with torch.no_grad():
            # alice_model -> paras = 0
            for model in params:
                for param in model:
                    param *= 0
            
            #  ????
            for model in models:
                model.get()
            
            # alice_model -> paras = new_params
            # new_params -> ?? -> alice_model
            for remote_index in range(len(compute_nodes)):
                for param_index in range(len(params[remote_index])):
                    params[remote_index][param_index].set_(new_params[param_index])
    return models

def test(models, test_loader):
    models[0].eval()
    test_loss = 0
    for data, target in test_loader:
        output = models[0](data)
        loss = nn.BCELoss()
        test_loss += loss(output.squeeze(), target)
        
    test_loss /= len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}\n'.format(test_loss))

## load data, set up virtual worker 

In [8]:
features = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
batch_size = 64
target = 'Survived'
lr = 0.01
train_size = 700

hook = sy.TorchHook(torch) 

# trianing, training data, model
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

# 
mpc_1 = sy.VirtualWorker(hook, id="mpc_1")
mpc_2 = sy.VirtualWorker(hook, id="mpc_2")
james = sy.VirtualWorker(hook, id="james")

In [ ]:
bob = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='bob', 
                            port=8778)
alice = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='alice', 
                            port=8777)

# 
mpc_1 = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='mpc1', 
                            port=8780)
mpc_2 = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='mpc2', 
                            port=8781)
james = WebsocketClientWorker(
                            host = 'localhost',
                            hook=hook,
                            id='james', 
                            port=8779)


In [9]:
compute_nodes = [bob, alice]

train_data = pd.read_csv('./data/train.csv')
train_data = train_data.fillna(0)
test_data = train_data[train_size:]
train_data = train_data[:train_size]
high_class_passager = train_data.loc[train_data.Pclass == 3]
low_class_passager = train_data.loc[train_data.Pclass.isin([1,2])]

training_data_size = min(len(high_class_passager), len(low_class_passager))
high_class_passager = high_class_passager[:training_data_size]
low_class_passager = low_class_passager[:training_data_size]


In [10]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,0,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,0,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,0,S


## process data set

In [11]:
remote_datasets = []
remote_datasets.append(send_data_to_worker(high_class_passager, features, target, alice))
remote_datasets.append(send_data_to_worker(low_class_passager, features, target, bob))

X_test = torch.from_numpy(test_data[features].values).float()
y_test = torch.from_numpy(test_data[target].values).float()
test_Dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_Dataset, batch_size=batch_size, shuffle=True)

## build torch model and dataset

In [12]:
# A Toy Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        return x


model = Net()

## initial model in workers

In [13]:
bobs_model = Net()
alices_model = Net()

bobs_optimizer = optim.SGD(bobs_model.parameters(), lr=lr)
alices_optimizer = optim.SGD(alices_model.parameters(), lr=lr)

models = [bobs_model, alices_model]
params = [list(bobs_model.parameters()), list(alices_model.parameters())]
optimizers = [bobs_optimizer, alices_optimizer]

In [14]:
for epoch in range(10):
    print(f"Epoch {epoch + 1}")
    models = train(epoch, compute_nodes, remote_datasets, models, mpc_1, mpc_2)
    test(models, test_loader)

Epoch 1
Test set: Average loss: 0.0195

Epoch 2
Test set: Average loss: 0.0104

Epoch 3
Test set: Average loss: 0.0100

Epoch 4
Test set: Average loss: 0.0100

Epoch 5
Test set: Average loss: 0.0095

Epoch 6
Test set: Average loss: 0.0099

Epoch 7
Test set: Average loss: 0.0093

Epoch 8
Test set: Average loss: 0.0099

Epoch 9
Test set: Average loss: 0.0091

Epoch 10
Test set: Average loss: 0.0094

